In [ ]:
!pip install rdkit

In [ ]:
!pip install xgboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import math

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import Descriptors3D
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/250k_rndm_zinc_drugs_clean_3.csv')

In [ ]:
df = df.drop(columns=['logP', 'qed', 'SAS'])

In [ ]:
df = df.iloc[130000:]

In [ ]:
df

,smiles
130000,Cc1cc(NC(=O)[C@H](Sc2cccc(F)c2)c2ccccc2)no1\n
130001,CCCN(CC(F)F)S(=O)(=O)c1cc([N+](=O)[O-])c(C)cc1...
130002,C[C@H]1CCN(C(=O)c2ccnc(OCC[NH+](C)C)c2)C[C@@H]...
130003,CCN1CCO[C@@H](CSc2nnnn2CCO)C1\n
130004,C[C@H](C(=O)[O-])[C@H](C)[NH+]1CC[C@H](N(C)C)C1\n
...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc...
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1\n
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1\n


In [ ]:
def get_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in mols:
        mol=Chem.AddHs(mol)
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

Mol_descriptors,desc_names = get_descriptors(df['smiles'])

Выходные данные были обрезаны до нескольких последних строк (5000).
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrContribs.size() == mol.getNumAtoms()
----------
Stacktrace:
----------
****

[22:34:26] 

****
Pre-condition Violation
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrContribs.size() == mol.getNumAtoms()
----------
Stacktrace:
----------
****

[22:34:26] 

****
Pre-condition Violation
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrContribs.size() == mol.getNumAtoms()
----------
S

KeyboardInterrupt: 

In [ ]:
df1 = pd.DataFrame(Mol_descriptors, columns=desc_names)
df1

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.888128,13.888128,1.009099,-4.897393,0.795315,51.120000,345.423,320.223,345.192117,134,...,0,0,0,0,0,0,0,0,0,0
1,14.611976,14.611976,1.012740,-4.148065,0.823310,23.416667,334.322,318.194,334.112899,126,...,0,0,0,0,0,0,0,0,0,0
2,13.723315,13.723315,0.596206,-3.677633,0.519575,22.222222,356.425,336.265,356.152478,134,...,0,0,0,0,0,0,0,0,0,0
3,13.848901,13.848901,0.594384,-3.658276,0.716191,20.541667,329.331,313.203,329.117570,124,...,0,0,0,0,0,0,0,0,0,0
4,13.619164,13.619164,0.180362,-4.550640,0.716788,30.000000,327.453,306.285,327.140533,120,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,14.147523,14.147523,0.628617,-4.391191,0.703235,43.285714,401.849,382.697,401.117508,144,...,0,0,0,0,0,0,0,0,0,0
29996,8.184194,8.184194,0.205552,-3.156123,0.723197,22.888889,260.322,248.226,260.073182,92,...,0,0,0,0,0,0,0,1,0,0
29997,8.062332,8.062332,0.107138,-3.336298,0.824405,25.500000,218.256,204.144,218.105528,84,...,0,0,0,0,0,0,0,0,0,0
29998,13.918942,13.918942,0.422020,-5.871548,0.633201,50.933333,450.566,426.374,450.126423,160,...,1,0,1,0,0,0,0,0,0,0


In [ ]:
df_new = pd.concat([df, df1], axis=1)

In [ ]:
col_to_drop = ['Ipc', 'smiles', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
               'BCUT2D_LOGPHI','BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']

In [ ]:
df_ml = df_new.drop(columns=col_to_drop)

In [ ]:
df_ml.to_csv('part5.csv')

In [ ]:
from joblib import dump, load

In [ ]:
stacking_4 = load('/content/drive/MyDrive/model_4.joblib')

In [ ]:
predicts = stacking_4.predict(df_ml)

In [ ]:
df['lgK'] = predicts

In [ ]:
df = df.sort_values(by=['lgK'], ascending=False)

In [ ]:
submit = df.head(500)['smiles']

In [ ]:
s = pd.DataFrame(df.head(500)['smiles']).reset_index()

In [ ]:
s = s.drop(columns=['index'])

In [ ]:
s

,smiles
0,CS(=O)(=O)N1CCN(C(=O)CN2C(=O)C(=O)N(C3CCCC3)C2...
1,O=C(NCCCN1C(=O)NC2(CCCC2)C1=O)Nc1cccc(CN2CCOC2...
2,CC(C)(C)OC(=O)N1CCC[C@@H](C(=O)N[C@@H]2CCCN(S(...
3,CC(C)OC(=O)[C@]1(C#N)[C@@H]2C=CC(C(N)=O)=CN2[C...
4,O=C(CN1C(=O)S/C(=C/c2ccc(-c3cccc(C(F)(F)F)c3)o...
...,...
495,C[C@]1(c2ccccc2Cl)NC(=O)N(CC(=O)NCCC2=CCCCC2)C...
496,CC(C)Nc1ncnc(NNC(=O)c2ccco2)c1[N+](=O)[O-]\n
497,CN(C1CCCCC1)S(=O)(=O)c1ccc(C(=O)Nc2ccc3c(c2)n(...
498,CC(C)[NH+]1CCC(N2CCN(CC3=c4ccccc4=[NH+]C3)C[C@...


In [ ]:
s.to_csv('submit2.csv', index=False)